In [10]:
import numpy as np
import pandas as pd
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()

from datetime import datetime

from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [11]:
n_years = 5

In [12]:
end_time = datetime.now()
start_time = datetime(
    end_time.year - n_years,
    end_time.month,
    end_time.day
)

In [13]:
def get_train_data(stock_name, column, start, end, scaler, fit=False):
    print(f'downloading {stock_name}')
    df = pdr.get_data_yahoo(
        stock_name,
        start,
        end
    )
    data = df.filter([column])
    dataset = data.values
    
    scaled_data = scaler.fit_transform(dataset) if fit else scaler.transform(dataset)

    X = []
    y = []

    for i in range(60, len(scaled_data)):
        X.append(scaled_data[i-60:i, 0])
        y.append(scaled_data[i, 0])
    
    return X, y

In [14]:
def get_aggregated_train_data(train_stocks, column, start_time, end_time):
    scaler = MinMaxScaler(feature_range=(0,1))

    X_train = []
    y_train = []

    for i, stock in enumerate(train_stocks):
        try:
            X, y = get_train_data(stock, column, start_time, end_time, scaler, fit=(i==0))
            X_train += X
            y_train += y
        except:
            continue
    
    X_train = np.array(X_train)
    y_train = np.array(y_train)

    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    with open(f'saved/scaler_{column}.pickle', 'wb') as pkl:
        pickle.dump(scaler, pkl)

    return X_train, y_train

In [15]:
def compile_save_model(column, X_train, y_train):
    model = Sequential([
        LSTM(128, return_sequences=True, input_shape= (X_train.shape[1], 1)),
        LSTM(64, return_sequences=True),
        LSTM(64, return_sequences=False),
        Dense(25),
        Dense(25),
        Dense(1)
    ])

    model.compile(optimizer='adam', loss='mean_squared_error')

    model.fit(X_train, y_train, batch_size=1, epochs=1)
    
    model.save(f'saved/model_{column}.h5', save_format='h5')

## Training All Models

In [16]:
train_stocks = [
    'AAPL',
    'BIVI',
    'GOOG',
    'NVMI',
    'CHRD',
    'ASMB',
    'GOGO',
    'CAFG',
    'BVXV',
    'AIBBU',
    'LGVCW',
    'AMZN'
]
columns = [
    'Open',
    'High',
    'Low',
    'Close',
    'Adj Close',
    'Volume'
]

In [17]:
for col in columns:
    print(f'doing {col}')
    X_train, y_train = get_aggregated_train_data(train_stocks, col, start_time, end_time)
    compile_save_model(col, X_train, y_train)
    print(f'{col} done')

doing Open
downloading AAPL


[*********************100%***********************]  1 of 1 completed
downloading BIVI
[*********************100%***********************]  1 of 1 completed
downloading GOOG
[*********************100%***********************]  1 of 1 completed
downloading NVMI
[*********************100%***********************]  1 of 1 completed
downloading CHRD
[*********************100%***********************]  1 of 1 completed
downloading ASMB
[*********************100%***********************]  1 of 1 completed
downloading GOGO
[*********************100%***********************]  1 of 1 completed
downloading CAFG
[*********************100%***********************]  1 of 1 completed
downloading BVXV
[*********************100%***********************]  1 of 1 completed
downloading AIBBU
[*********************100%***********************]  1 of 1 completed
downloading LGVCW
[*********************100%***********************]  1 of 1 completed


2023-07-01 02:00:38.352152: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-01 02:00:38.358221: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-01 02:00:38.360288: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

9329/9329 [==============================] - 470s 50ms/step - loss: 0.0107
Open done
doing High
downloading AAPL
[*********************100%***********************]  1 of 1 completed
downloading BIVI
[*********************100%***********************]  1 of 1 completed
downloading GOOG
[*********************100%***********************]  1 of 1 completed
downloading NVMI
[*********************100%***********************]  1 of 1 completed
downloading CHRD
[*********************100%***********************]  1 of 1 completed
downloading ASMB
[*********************100%***********************]  1 of 1 completed
downloading GOGO
[*********************100%***********************]  1 of 1 completed
downloading CAFG
[*********************100%***********************]  1 of 1 completed
downloading BVXV
[*********************100%***********************]  1 of 1 completed
downloading AIBBU
[*********************100%***********************]  1 of 1 completed
downloading LGVCW
[*********************100

2023-07-01 02:08:36.188242: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-01 02:08:36.192423: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-01 02:08:36.194428: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

...

9329/9329 [==============================] - 443s 47ms/step - loss: 0.0115
Close done
doing Adj Close
downloading AAPL
[*********************100%***********************]  1 of 1 completed
downloading BIVI
[*********************100%***********************]  1 of 1 completed
downloading GOOG
[*********************100%***********************]  1 of 1 completed
downloading NVMI
[*********************100%***********************]  1 of 1 completed
downloading CHRD
[*********************100%***********************]  1 of 1 completed
downloading ASMB
[*********************100%***********************]  1 of 1 completed
downloading GOGO
[*********************100%***********************]  1 of 1 completed
downloading CAFG
[*********************100%***********************]  1 of 1 completed
downloading BVXV
[*********************100%***********************]  1 of 1 completed
downloading AIBBU
[*********************100%***********************]  1 of 1 completed
downloading LGVCW
[******************

2023-07-01 02:31:09.144563: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-01 02:31:09.146231: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-01 02:31:09.149749: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

9329/9329 [==============================] - 441s 47ms/step - loss: 0.0115
Adj Close done
doing Volume
downloading AAPL
[*********************100%***********************]  1 of 1 completed
downloading BIVI
[*********************100%***********************]  1 of 1 completed
downloading GOOG
[*********************100%***********************]  1 of 1 completed
downloading NVMI
[*********************100%***********************]  1 of 1 completed
downloading CHRD
[*********************100%***********************]  1 of 1 completed
downloading ASMB
[*********************100%***********************]  1 of 1 completed
downloading GOGO
[*********************100%***********************]  1 of 1 completed
downloading CAFG
[*********************100%***********************]  1 of 1 completed
downloading BVXV
[*********************100%***********************]  1 of 1 completed
downloading AIBBU
[*********************100%***********************]  1 of 1 completed
downloading LGVCW
[*****************

2023-07-01 02:38:37.524906: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-01 02:38:37.527729: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-01 02:38:37.529492: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

9329/9329 [==============================] - 444s 47ms/step - loss: 0.0030
Volume done
